## BRONZE TO SILVER LAYER

### GOLD LAYER - PROCESS HOLDING RECORDS HISTORY


In [8]:
# Import necessary libraries and utility functions
import pandas as pd
from common_utilities import global_path, logger
from datetime import datetime

df = pd.read_csv(global_path.holdings_gold_file_path)
df["datetime"] = pd.to_datetime(df["datetime"])
df["date"] = df["datetime"].dt.date

df = df.merge(df.groupby(["date", "stock_name"])["datetime"].max().reset_index(), on=["date", "stock_name", "datetime"]).reset_index(drop=True)

# Process each stock name separately
result = []
for stock_name in df["stock_name"].unique():
    stock_data = df[df["stock_name"] == stock_name].copy()

    # Set 'date' as index and drop duplicate indices, keeping the first occurrence
    stock_data = stock_data.set_index("date")
    stock_data = stock_data[~stock_data.index.duplicated(keep="last")]

    # Ensure there are no duplicate dates before reindexing
    date_range = pd.date_range(
        start=stock_data.index.min(),
        end=datetime.today().date(),
        freq="D",
    )

    # Reindex to include all dates and forward fill missing values
    stock_data = stock_data.reindex(date_range, method="ffill")

    # Reset index to bring 'date' back as a column
    stock_data = stock_data.reset_index().rename(columns={"index": "date"})
    result.append(stock_data)

# Combine processed DataFrames into one

df = pd.concat(result, ignore_index=True)

# Load stock prices and merge with the main DataFrame
df_StockPrice = pd.read_csv(global_path.stockprice_silver_file_path)
df_StockPrice["date"] = pd.to_datetime(df_StockPrice["date"])
logger.info(
    f"Loaded SILVER Layer stock price data from: {global_path.stockprice_silver_file_path}"
)

# Merge stock price data
df = pd.merge(df, df_StockPrice, on=["date", "stock_name"], how="left")

df["current_value"] = df["ltp"] * df["holding_quantity"]
# Calculate PnL and percentage
df["pnl_amount"] = df["current_value"] - df["holding_amount"]
df["pnl_percentage"] = (df["pnl_amount"] / df["holding_amount"]) * 100
df = df[(df["holding_amount"] != 0) & (df["current_value"] != 0)]
df = df.round(2)

# Final sorting and column selection

df = df.sort_values(by=["segment", "stock_name", "date"]).reset_index(drop=True)

# Save the final DataFrame to a CSV file

df = df[
    [
        "date",
        "segment",
        "stock_name",
        "holding_quantity",
        "holding_price_avg",
        "holding_amount",
        "ltp",
        "current_value",
        "pnl_amount",
        "pnl_percentage",
    ]
]
df.to_csv(global_path.holdings_gold_file_path_v2, index=None)
logger.info("GOLD Layer CSV file for Holdings successfully created at:")
logger.info(global_path.holdings_gold_file_path_v2.resolve())
# Display DataFrame information and print success message
df.info()

2024-08-05T19:57:27Z - INFO - Loaded SILVER Layer stock price data from: C:\Users\prashant.tripathi\Code\Upstox\DATA\SILVER\StockPrice\StockPrice_data.csv
2024-08-05T19:57:27Z - INFO - GOLD Layer CSV file for Holdings successfully created at:
2024-08-05T19:57:27Z - INFO - C:\Users\prashant.tripathi\Code\Upstox\DATA\GOLD\Holdings\Holdings_datav2.csv


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5929 entries, 0 to 5928
Data columns (total 10 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   date               5929 non-null   datetime64[ns]
 1   segment            5929 non-null   object        
 2   stock_name         5929 non-null   object        
 3   holding_quantity   5929 non-null   float64       
 4   holding_price_avg  5929 non-null   float64       
 5   holding_amount     5929 non-null   float64       
 6   ltp                3827 non-null   float64       
 7   current_value      3827 non-null   float64       
 8   pnl_amount         3827 non-null   float64       
 9   pnl_percentage     3827 non-null   float64       
dtypes: datetime64[ns](1), float64(7), object(2)
memory usage: 463.3+ KB


In [9]:
df[df['date'].dt.date == datetime.today().date()]

,date,segment,stock_name,holding_quantity,holding_price_avg,holding_amount,ltp,current_value,pnl_amount,pnl_percentage
975,2024-08-05,EQ,IRCTC,3.00,896.20,2688.60,NaN,NaN,NaN,NaN
1029,2024-08-05,EQ,KPITTECH,2.00,1500.00,3000.00,NaN,NaN,NaN,NaN
3577,2024-08-05,EQ,TATAMOTORS,100.00,261.00,26100.40,NaN,NaN,NaN,NaN
4852,2024-08-05,EQ,TATAPOWER,150.00,147.49,22124.00,NaN,NaN,NaN,NaN
5772,2024-08-05,FO,NIFTY-CE-24400-11JUL2024,0.00,0.00,2820.00,NaN,NaN,NaN,NaN
5778,2024-08-05,FO,NIFTY-CE-24700-08AUG2024,25.00,140.00,3500.00,NaN,NaN,NaN,NaN
5854,2024-08-05,MF,MIRAE-ASSET-TAX-SAVER-DIRECT-GROWTH,39.80,50.26,2000.27,NaN,NaN,NaN,NaN
5928,2024-08-05,MF,SBI-MAGNUM-TAXGAIN-SCHEME-DIR-GR,4.53,441.62,2000.56,NaN,NaN,NaN,NaN


In [10]:
# Read the CSV file
df_trade_history = pd.read_csv(global_path.tradehistory_silver_file_path)

df_trade_history["quantity"] = df_trade_history.apply(
    lambda row: row["quantity"] if row["side"] == "BUY" else row["quantity"] * (-1),
    axis=1
)

df_trade_history = df_trade_history.groupby(["exchange","segment","stock_name"])["quantity"].sum().reset_index()
df_trade_history = df_trade_history[df_trade_history["quantity"]!= 0]
df_trade_history = df_trade_history.sort_values(by=["exchange","segment","stock_name"]).reset_index(drop=True)
df_trade_history


,exchange,segment,stock_name,quantity
0,BMF,MF,MIRAE-ASSET-TAX-SAVER-DIRECT-GROWTH,39.80
1,BMF,MF,SBI-MAGNUM-TAXGAIN-SCHEME-DIR-GR,4.53
2,FON,FO,NIFTY-CE-24700-08AUG2024,25.00
3,FON,FO,NIFTY-PE-24650-18JUL2024,100.00
4,NSE,EQ,IRCTC,3.00
5,NSE,EQ,KPITTECH,2.00
6,NSE,EQ,TATAMOTORS,100.00
7,NSE,EQ,TATAPOWER,150.00
